In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.nn import Sequential, Linear

In [2]:
# Loading Dataset
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [4]:
# Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [6]:
# Data loading class
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels) -> None:
        super().__init__()
        self.features = torch.tensor(features, dtype = torch.float32)
        self.labels = torch.tensor(labels, dtype = torch.long)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [7]:
# Creating an object
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Data Loaders
train_loader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True, pin_memory = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = True, pin_memory = True)

In [8]:
import torch.nn.init as init # Used to initialize the parameters
class CustomModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            # Just trying multiple activation functions
            nn.Linear(in_features = num_features, out_features = 128),
            nn.LeakyReLU(negative_slope = 0.01),

            nn.Linear(in_features = 128, out_features = 64),
            nn.ELU(alpha = 1),

            nn.Linear(in_features = 64, out_features = 32),
            nn.PReLU(num_parameters = 1, init = 0.25),

            nn.Linear(in_features = 32, out_features = 10),
            nn.Sigmoid()
        )

    def initialize_weights(self, layer):
        if isinstance(layer, nn.Linear):
            # init contains utilities for initializing the weights
            init.xavier_normal_(layer.weight) # xavier normal initialization - Inplace operation
        elif isinstance(layer, nn.Conv2d):
            init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu') # He Initialization

    def forward(self, X):
        self.network.apply(self.initialize_weights)
        return self.network(X)

In [9]:
# Compiling Model
learning_rate = 0.01
epochs = 10

# Model Object
model = CustomModel(num_features = X_train.shape[1])

# Defining Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)

In [10]:
# Training Loop
for epoch in range(epochs):
    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:
        # Forward pass
        y_pred = model(batch_features)

        # Calculate loss
        loss = criterion(y_pred, batch_labels)

        # Zero Gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Update Weights
        optimizer.step()

        # Epoch loss
        total_epoch_loss += loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 2.2993642330169677
Epoch: 2 , Loss: 2.2366551558176675
Epoch: 3 , Loss: 2.184143813451131
Epoch: 4 , Loss: 2.1150835434595745
Epoch: 5 , Loss: 2.081625970204671
Epoch: 6 , Loss: 2.0460004091262816
Epoch: 7 , Loss: 1.9759919087092082
Epoch: 8 , Loss: 1.9380910317103068
Epoch: 9 , Loss: 1.928854481379191
Epoch: 10 , Loss: 1.8939714670181274


In [11]:
model.eval()

CustomModel(
  (network): Sequential(
    (0): Linear(in_features=30, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ELU(alpha=1)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): PReLU(num_parameters=1)
    (6): Linear(in_features=32, out_features=10, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
# Evaluating on Test Data - Demo
from torch import tensor
from torchmetrics.classification import MulticlassAccuracy
target = tensor([2, 1, 0, 0])
preds = tensor([2, 1, 0, 1])
metric = MulticlassAccuracy(num_classes=3)
metric(preds, target)

tensor(0.8333)

In [14]:
mca = MulticlassAccuracy(num_classes=3, average=None)
mca(preds, target)

tensor([0.5000, 1.0000, 1.0000])

In [15]:
# Evaluating on Test Data
accuracy = []
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        y_pred = model(batch_features)
        _, predicted = torch.max(y_pred, 1)
        metric = MulticlassAccuracy(num_classes = 3)
        accuracy.append(metric(predicted, batch_labels))
print(sum(accuracy) / len(accuracy))

tensor(0.6123)
